#Práctica 3: CNN.
##Introducción
Una CNN (Convolutional Neural Network, o Red Neuronal Convolucional) es un tipo de arquitectura de red neuronal usualmente utilizada en tareas relacionadas con el análisis de datos espaciales o estructurados, como imágenes y videos. Las CNNs están diseñadas para aprovechar las estructuras locales de los datos y son especialmente efectivas en la detección de patrones como bordes, texturas, y formas en imágenes. Por su parte, el dataset seleccionado para la realización de esta práctica consiste en un gran numero de imágenes segmentadas de núcleos biológicos pertenecientes a células de una amplia variedad de contextos.
##Objetivo
####En esta práctica se implementará una red neuronal de tipo CNN para el análisis de un dataset que contiene datos de núcleos biológicos.
#Metodología

###Carga de librerías.
####En esta primera parte del código se importaron las librerías necesarias para el desarrollo del código. La librería "os" para manejo del sistema operativo, "numpy" para operaciones numéricas, "tensorflow" para construir y entrenar la red neuronal, y "matplotlib" para graficar resultados.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

ImportError: Matplotlib requires numpy>=1.20; you have 1.19.2

###Definición de rutas y parámetros
####Aquí se definen las rutas a los directorios de entrenamiento y validación. Además, se establecen las dimensiones de las imágenes (IMG_HEIGHT y IMG_WIDTH) y el tamaño del lote (BATCH_SIZE).

In [ ]:
# Definir las rutas a los directorios de entrenamiento y validación
train_dir = '/mnt/c/Users/Usuario/Desktop/entorno/l'
validation_dir = '/mnt/c/Users/Usuario/Desktop/entorno/s'

# Definir el tamaño de las imágenes y el tamaño del lote
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32

###Generadores de datos
####Se crearon los generadores de datos para cargar y preprocesar las imágenes, escalando los valores de los píxeles a un rango de 0 a 1. flow_from_directory carga imágenes de los directorios especificados y ajusta su tamaño.

In [ ]:
# Crear generadores de datos para cargar y preprocesar las imágenes
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)

val_data_gen = validation_image_generator.flow_from_directory(
    batch_size=BATCH_SIZE,
    directory=validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary'
)

###Definición de la arquitectura de la CNN
####°Capas convolucionales (Conv2D): Detectan características en las imágenes usando filtros.
####°Capas de agrupamiento (MaxPooling2D): Reducen la dimensionalidad.
####°Aplanado (Flatten): Convierte la matriz 2D resultante de las capas convolucionales en un vector 1D.
####°Capas densas (Dense): Realizan la clasificación final. La última capa utiliza la función de activación sigmoid para salida binaria.

In [ ]:
# Definir la arquitectura de la CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

###Compilación y resumen del modelo.
####Compila el modelo especificando el optimizador (adam), la función de pérdida (binary_crossentropy), y las métricas (accuracy). Luego se muestra un resumen del modelo.

In [ ]:
# Compilar el modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Resumen del modelo
model.summary()

###Entrenamiento del modelo.
####Se entrena el modelo utilizando los generadores de datos de entrenamiento y validación. steps_per_epoch y validation_steps especifican el número de pasos por época. epochs define el número de épocas de entrenamiento.

In [ ]:
# Entrenar el modelo
history = model.fit(
    train_data_gen,
    steps_per_epoch=train_data_gen.samples // BATCH_SIZE,
    epochs=15,
    validation_data=val_data_gen,
    validation_steps=val_data_gen.samples // BATCH_SIZE
)

###Evaluación del modelo.
####Se evalúa el rendimiento del modelo en los datos de validación y se imprime la pérdida (loss) y la precisión (accuracy).

In [ ]:
# Evaluar el modelo
loss, accuracy = model.evaluate(val_data_gen)
print(f'Loss: {loss}, Accuracy: {accuracy}')

###Graficación de los resultados.
####Extaccion de los datos del historial y definición del rango de épocas.
#####El objeto history contiene el historial de entrenamiento del modelo. history.history es un diccionario que almacena las listas de valores de precisión (accuracy), precisión de validación (val_accuracy), pérdida (loss) y pérdida de validación (val_loss) para cada época del entrenamiento.

In [ ]:
# Graficar la precisión y la pérdida durante el entrenamiento
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(15)

###Creación de los gráficos.
####Se hacen las gráficas de precisión y pérdida. La gráfica de Precisión (Accuracy) permite ver cómo la precisión del modelo en los datos de entrenamiento y validación cambia a lo largo de las épocas. Por su parte, la gráfica de Pérdida (Loss) muestra cómo la función de pérdida del modelo se comporta durante el entrenamiento. 

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()